## Mapping Work Locations with 1000+ Employees using Folium

In [1]:
import pathlib
import pandas as pd
import numpy as np
import geopandas as gp
import shapely.geometry as geom
import folium
from folium.plugins import MarkerCluster

#### Set input file location and read into Pandas

In [2]:
input_file = 'od_stats.csv.gz'
ODpath = pathlib.Path('../data/OD/')
ODfile = ODpath.joinpath(input_file)

with ODfile.open(mode='r') as fid:
#    df_All = pd.read_csv(ODfile, dtype = {'w_geocode': object, 'h_geocode': object})
    df_All = pd.read_csv(ODfile, dtype = {'w_geocode': object})

print ('input file: ',ODfile)
print('\nrecords loaded to dataframe:', "{:,}".format(len(df_All)),'\n\n')
df_All.head(1)

input file:  ../data/OD/od_stats.csv.gz

records loaded to dataframe: 17,233 




,w_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon
0,010010205001001,1100,536,426,138,534,412,154,13,540,547,32.45674,-86.415025


#### Set the state to analyze

In [3]:
state_id = '21'

#### Set the output file location for html

In [4]:
output_loc = pathlib.Path('../docs/')
output_loc.mkdir(exist_ok=True, parents=True)
print ('path for html output:',output_loc)

path for html output: ../docs


#### Drop all records where the work location does not being with the chosen state_id

In [5]:
df = df_All.drop(df_All[~df_All['w_geocode'].str.startswith(state_id)].index)
df = df.reset_index(drop=True)
print('\nnew dataframe length: ', "{:,}".format(len(df)),'\n\n')
df.head(1)


new dataframe length:  223 




,w_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon
0,210099505002000,1238,137,854,247,71,599,568,0,0,1238,37.010798,-85.906837


#### Create a column for the dominant industry sector of the block

In [6]:
df['industry_max']=df[['SI01','SI02','SI03']].idxmax(axis=1)
df.head(1)

,w_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon,industry_max
0,210099505002000,1238,137,854,247,71,599,568,0,0,1238,37.010798,-85.906837,SI03


In [7]:
df.loc[df['industry_max'] == 'SI01', 'industry'] = 'Goods Producing'
df.loc[df['industry_max'] == 'SI02', 'industry'] = 'Trade, Transportation, Utilities'
df.loc[df['industry_max'] == 'SI03', 'industry'] = 'Service'
df.head(3)

,w_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon,industry_max,industry
0,210099505002000,1238,137,854,247,71,599,568,0,0,1238,37.010798,-85.906837,SI03,Service
1,210150702005017,1327,260,795,272,164,637,526,208,590,529,38.982623,-84.625940,SI02,"Trade, Transportation, Utilities"
2,210150703011003,1455,142,919,394,45,367,1043,1177,45,233,38.977810,-84.625726,SI01,Goods Producing


In [8]:
#df.loc[df['SI01'] == 'SI03', 'Industry'] = 'Service'
df2 = pd.DataFrame([df.w_geocode,df.SI01,df.SI02,df.SI03]).transpose()
df2 = df2.rename(columns={'w_geocode': 'GEOID','SI01': 'Goods Producing', 'SI02': 'Trade, Transportation, Utilities', 'SI03': 'Service'})
df2.head(10)

,GEOID,Goods Producing,"Trade, Transportation, Utilities",Service
0,210099505002000,0,0,1238
1,210150702005017,208,590,529
2,210150703011003,1177,45,233
3,210150703011004,1341,276,215
4,210150703011019,1394,95,68
5,210150703011029,370,1102,36
6,210150703111008,1513,1078,512
7,210150703111044,217,24,3474
8,210150703112018,5,84,959
9,210150703112041,1461,549,315


In [9]:
def color_icon(industry):
    if industry == 'SI01': 
        col='green'
    elif industry == 'SI02':
        col='red'
    else:
        col='blue'
    return col

In [10]:
def popup(block_id, industry, employed,si01,si02,si03):
    # generate the popup message that is shown on click.
    popup_text = "<br>GEOID: {} <br> Main industry type: {}<br> Number employed: {}<br> Goods Producing: {}<br> Trade, Transportation, Utilities: {}<br> Service: {}"
    popup_text = popup_text.format(block_id, industry,employed,si01,si02,si03)
    return popup_text

#### Create lat lon list for mapping locations with 1000+ employees

In [11]:
locations = df[['w_lat', 'w_lon']]
locationlist = locations.values.tolist()
print ('\nthe number of locations with 1000+ employees:',  "{:,}".format(len(locationlist)),'\n\n')


the number of locations with 1000+ employees: 223 




#### Generate map with basic markers

In [12]:
m = folium.Map(location=[37.645556, -84.769722], tiles='cartodbpositron',
                zoom_start=7, control_scale=True, prefer_canvas=True)

legend_html =   '''
                <div style="position: fixed; 
                            bottom: 50px; left: 5px; width: 270px; height: 140px; 
                            border:2px solid grey; z-index:9999; font-size:16px;
                            ">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>Industry Type </b><br>
                            &nbsp;&nbsp;<i class="fa fa-map-marker fa-2x" style="color:green"></i>&nbsp; Goods Producing &nbsp;<br>
                            &nbsp;&nbsp;<i class="fa fa-map-marker fa-2x" style="color:red"></i>&nbsp; Trade, Transportation, Utilities &nbsp;<br>
                            &nbsp;&nbsp;<i class="fa fa-map-marker fa-2x" style="color:blue"></i>&nbsp; Service &nbsp; 
                </div>
                ''' 

title_html =   '''
                <div style="position: fixed; 
                            top: 10px; left: 450px; width: 500px; height: 30px; 
                            border:0px solid grey; z-index:9999; font-size:24px;
                            "><b>Block Locations with 1000+ Employees </b><br>
                </div>
                ''' 

m.get_root().html.add_child(folium.Element(title_html))    

for point in range(len(locationlist)):
    #folium.Marker(locations[point], popup=df_blocks['w_geocode'][point]).add_to(m)
    folium.Marker(locationlist[point],popup=folium.Popup(popup(df['w_geocode'][point],df['industry'][point],
                                                               df['S000'][point],df['SI01'][point],df['SI02'][point],df['SI03'][point]),
                  max_width=650),
                  icon= folium.Icon(color=color_icon(df['industry_max'][point]))).add_to(m)
    
m.get_root().html.add_child(folium.Element(legend_html))  

m.save('../docs/basic_markers.html')


__[Click here for the interactive map](https://markcruse.github.io/698-final-project/basic_markers.html)__

#### Generate map of circle markers with radius based on # of employees

In [13]:
# generate a new map
folium_map = folium.Map(location=[37.645556, -84.769722],
                        zoom_start=8,
                        tiles="CartoDB dark_matter",
                        width='100%')
legend_html =   '''
                <div style="position: fixed; 
                            bottom: 50px; left: 5px; width: 270px; height: 140px; 
                            border:2px solid grey; z-index:9999; font-size:16px;color:lightgrey;
                            ">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>Industry Type </b><br>
                            &nbsp;&nbsp;<i class="fa fa-map-marker fa-2x" style="color:green"></i>&nbsp; Goods Producing &nbsp;<br>
                            &nbsp;&nbsp;<i class="fa fa-map-marker fa-2x" style="color:red"></i>&nbsp; Trade, Transportation, Utilities &nbsp;<br>
                            &nbsp;&nbsp;<i class="fa fa-map-marker fa-2x" style="color:blue"></i>&nbsp; Service &nbsp; 
                </div>
                ''' 

title_html =   '''
                <div style="position: fixed; 
                            top: 10px; left: 450px; width: 500px; height: 30px; 
                            border:0px solid grey; z-index:9999; font-size:24px; color:lightgrey;
                            "><b>Block Locations with 1000+ Employees </b><br>
                </div>
                ''' 

folium_map.get_root().html.add_child(folium.Element(title_html))    

# for each row in the data, add a cicle marker
for index, row in df.iterrows():
    # generate the popup message that is shown on click.
    popup_text = "<br>GEOID: {} <br> Main industry type: {}<br> Number employed: {}<br> Goods Producing: {}<br> Trade, Transportation, Utilities: {}<br> Service: {}"
    popup_text = popup_text.format(row['w_geocode'],row['industry'],row['S000'],row['SI01'],row['SI02'],row['SI03'])
    

    # radius of circles
    radius = row.S000/300

    # choose the color of the marker
    if row['industry_max'] == 'SI01': 
        color='green'
    elif row['industry_max'] == 'SI02':
        # color="#FFCE00" # orange
        # color="#007849" # green
        color="#E37222" # tangerine
    else:
        # color="#0375B4" # blue
        # color="#FFCE00" # yellow            
        color="#0A8A9F" # teal

    # add marker to the map
    folium.CircleMarker(location=(row['w_lat'],
                                  row['w_lon']),
                        radius=radius,
                        color=color,
                        popup=folium.Popup(popup_text,max_width=650)).add_to(folium_map)
    
folium_map.get_root().html.add_child(folium.Element(legend_html)) 


folium_map.save('../docs/circle_markers.html')


#folium_map

__[Click here for the interactive map](https://markcruse.github.io/698-final-project/circle_markers.html)__

#### Generate map of clustered markers

In [14]:
m = folium.Map(location=[37.645556, -84.769722], tiles='cartodbpositron',
                zoom_start=7, control_scale=True, prefer_canvas=True)

marker_cluster = MarkerCluster().add_to(m)


legend_html =   '''
                <div style="position: fixed; 
                            bottom: 50px; left: 5px; width: 270px; height: 140px; 
                            border:2px solid grey; z-index:9999; font-size:16px;
                            ">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>Industry Type </b><br>
                            &nbsp;&nbsp;<i class="fa fa-map-marker fa-2x" style="color:green"></i>&nbsp; Goods Producing &nbsp;<br>
                            &nbsp;&nbsp;<i class="fa fa-map-marker fa-2x" style="color:red"></i>&nbsp; Trade, Transportation, Utilities &nbsp;<br>
                            &nbsp;&nbsp;<i class="fa fa-map-marker fa-2x" style="color:blue"></i>&nbsp; Service &nbsp; 
                </div>
                ''' 

title_html =   '''
                <div style="position: fixed; 
                            top: 10px; left: 450px; width: 500px; height: 30px; 
                            border:0px solid grey; z-index:9999; font-size:24px;
                            "><b>Block Locations with 1000+ Employees </b><br>
                </div>
                ''' 
        
m.get_root().html.add_child(folium.Element(title_html))    

for point in range(len(locationlist)):
    #folium.Marker(locations[point], popup=df_blocks['w_geocode'][point]).add_to(m)
    folium.Marker(locationlist[point],popup=folium.Popup(popup(df['w_geocode'][point],df['industry'][point],
                                                               df['S000'][point],df['SI01'][point],df['SI02'][point],df['SI03'][point]),
                  max_width=650),
                  icon= folium.Icon(color=color_icon(df['industry_max'][point]))).add_to(marker_cluster)
    
    
m.get_root().html.add_child(folium.Element(legend_html))  

m.save('../docs/cluster_markers.html')

#m

__[Click here for the interactive map](https://markcruse.github.io/698-final-project/cluster_markers.html)__